# Fuentes

### Link: https://www.dlology.com/blog/how-to-do-hyperparameter-search-with-baysian-optimization-for-keras-model/
Explicación de cómo utilizar el método de optimización bayesiana para buscar los hiper parámetros del modelo para resolver el problema.

### Link: https://www.kaggle.com/prashant111/bayesian-optimization-using-hyperopt
Ahora viendo cómo resolverlo utilizando la librería de **hyperopt**.

# 1. Cargando base de datos

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import importlib

In [3]:
import sys

In [4]:
sys.path.insert(0, '..')

In [5]:
sys.path.insert(0, '../..')

In [6]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [7]:
from sklearn import preprocessing

## 2.1. Codificación de variables no numéricas o categóricas

In [8]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [9]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [10]:
# Create a label encoder for the region variable or feature and create a new column in the dataframe
# with the encoded version of the region
region_encoder = preprocessing.LabelEncoder()
region_encoder.fit(df['region'])
df['region-encoded'] = region_encoder.transform(df['region'])

In [11]:
df.head()

,age,sex,bmi,children,smoker,region,charges,sex-encoded,smoker-encoded,region-encoded
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,3
1,18,male,33.770,1,no,southeast,1725.55230,1,0,2
2,28,male,33.000,3,no,southeast,4449.46200,1,0,2
3,33,male,22.705,0,no,northwest,21984.47061,1,0,1
4,32,male,28.880,0,no,northwest,3866.85520,1,0,1


## 2.2. Filtrado de variables

In [12]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'region-encoded']]
df_y = df['charges']

# 3. Separación del conjunto de entrenamiento y evaluación

In [13]:
from sklearn import model_selection

In [14]:
from sklearn import preprocessing

## 3.1. Separación de los conjuntos
Es importante notar que, se realiza la separación del conjunto de datos original en **train**, **valid** y **test**, por fuera del framework de Keras para garantizar un adecuado tratamiento de los conjuntos acorde a la metodología empleada. En otras palabras, de esta forma nos aseguramos que cualquier preprocesamiento o normalización sobre validación (valid) y evaluación (test) se realiza a partir de la información obtenida en entrenamiento.

In [15]:
# Split the dataset into train_valid and test
x_train_valid, x_test, y_train_valid, y_test = model_selection.train_test_split(df_x, df_y, test_size=0.2, random_state=5, shuffle=True)

In [16]:
# Split the dataset into train and valid
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=5, shuffle=True)

## 3.2. Normalización de variables

In [17]:
# Select the variables where the z-score will be applied
scalable_variables = ['bmi', 'age']

if scalable_variables:
    # Create an instance of the StandardScaler for each variable
    scaler = preprocessing.StandardScaler()

    # Fit the distribution
    scaler.fit(x_train.loc[:, scalable_variables])

    # Transform and normalize all variables
    x_train.loc[:, scalable_variables] = scaler.transform(x_train.loc[:, scalable_variables])
    x_valid.loc[:, scalable_variables] = scaler.transform(x_valid.loc[:, scalable_variables])
    x_test.loc[:, scalable_variables] = scaler.transform(x_test.loc[:, scalable_variables])

## 3.3. Conjuntos de entrenamiento, validación y evaluación
Particularmente, para las redes neuronales de múltiples capas, en este problema vamos a estar utilizando una capa de *embedding* para poder reducir la dimensionalidad necesaria para desacoplar las categorías de la variable *region*. Es decir, normalmente podríamos utilizar un *one hot encoding* para independizar en diferentes variables, pero produce una mayor dimensionalidad y poca interpretabilidad de lo que la red neuronal aprende durante el entrenamiento. En conclusión, debemos separar los grupos de entradas por el formato de entrada de la red neuronal.

In [18]:
x_train = [x_train[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_train['region-encoded']]
x_valid = [x_valid[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_valid['region-encoded']]
x_test = [x_test[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded']], x_test['region-encoded']]

# 4. Multilayer Perceptron

In [19]:
import hyperopt

In [20]:
from hyperopt.pyll import scope

In [21]:
from src import mlp_helper
importlib.reload(mlp_helper);

## 4.1. Creación de la función **objetivo**
Creamos la función **objetivo** la cual recibe los parámetros a partir de los cuales se crea el modelo, se lo entrena y se evalúa su performance.

In [22]:
def objective(kwargs):
    """ Objective function for the hyperopt algorithm. """
    
    # Setting some fixed values
    kwargs['tensorboard_on'] = False
    kwargs['summary_on'] = False
    kwargs['epochs'] = 500
    kwargs['batch_size'] = 64
    kwargs['optimizer'] = 'adam'
    kwargs['beta_1'] = 0.9
    kwargs['beta_2'] = 0.999
    
    # Running the model with the given hyperparameters and retrieving the test set performance
    # WARNING! We're using the same valid set for both valid and test, but can be ignored, it does not affect
    # because it was something we had to do to reuse the function...
    _, _, mae_test = mlp_helper.run_model(x_train, y_train, x_valid, y_valid, x_valid, y_valid, **kwargs)
    return mae_test

## 4.2. Creación del **espacio** de hiperparámetros
Creamos el **espacio** en el cual se encuentran cada uno de los hiperparáemtros que queremos probar, es decir, cuáles son los hiperparámetros y qué valores pueden tomar en cada prueba a realizar por el optimizador de hiperparámetros.

In [23]:
space = {
    'use_batch_normalization': hyperopt.hp.choice('use_batch_normalization', [False, True]),
    'learning_rate': hyperopt.hp.choice('learning_rate', [1.0, 0.5, 0.1, 0.01]),
    'hidden_layers': scope.int(hyperopt.hp.quniform('hidden_layers', 0, 10, 1)),
    'units_per_layer': scope.int(hyperopt.hp.quniform('units_per_layer', 2, 100, 10)),
    'hidden_layer_activation': hyperopt.hp.choice('hidden_layer_activation', ['relu', 'elu'])
}

## 4.3. Selección del **algoritmo** de optimización
Se puede elegir un **algoritmo** de optimización para el proceso de búsqueda de los mejores hiperparámetros dentro del espacio consignado acorde a los resultados que se van obteniendo de la función objetivo.

In [24]:
algorithm = hyperopt.tpe.suggest

## 4.4. Ejecutando la búsqueda
Se ejecuta la búsqueda de los mejores hiperparámetros.

In [25]:
best = hyperopt.fmin(objective, space, algo=algorithm, max_evals=100)

100%|███████████| 100/100 [43:22<00:00, 26.03s/trial, best loss: 1656.63330078125]


In [26]:
best

{'hidden_layer_activation': 0,
 'hidden_layers': 7.0,
 'learning_rate': 1,
 'units_per_layer': 80.0,
 'use_batch_normalization': 1}